##XP-EHH anlysis

In [ ]:
!pip install -qq malariagen_data
import malariagen_data
import numpy as np
import pandas as pd
import allel
import zarr
import matplotlib.pyplot as plt
import seaborn as sns

ag3 = malariagen_data.Ag3()
ag3

In [ ]:
# Contigs and cohorts setup
#GSS with iHs
def run_XP_EHH(taxon1, taxon2, contig,  country, window_size=200):
  """
  function that compute xp_ehh value for between twon taxon for a given contig
  """
  try:
    pos, xp_ehh = ag3.xpehh_gwss(
            contig=contig,
            cohort1_query = f"country=='{country}' and taxon=='{taxon1}'",
            cohort2_query = f"country=='{country}' and taxon=='{taxon2}'",
            window_size=window_size,
            analysis="gamb_colu",
    )
    return pos, xp_ehh # x: ndarray xp_ehh: ndarray
  except Exception as e:
    print(f"Error running XP-EHH: {e}")
    return None, None

def run_XP_EHH_loc(taxon, cohort1, cohort2, country ,contig, window_size=200):
  """
  function that compute xp_ehh value for between twon subpopulation for a given contig
  """
  list_cohort_group = ['country_iso', 'admin1_name', 'admin1_iso', 'admin2_name', 'cohort_admin1_year']
  print("cohort group list:")
  for i in list_cohort_group:
    print(i)
  cohort_group = input("Enter the cohort group to use: ")
  if cohort_group not in list_cohort_group or cohort_group is None:
    print("Invalid cohort group. Please choose from the list.")
    return
  try:
    pos, xp_ehh = ag3.xpehh_gwss(
            contig=contig,
            cohort1_query = f"country=='{country}' and taxon=='{taxon}' and {cohort_group} == '{cohort1}'",
            cohort2_query = f"country=='{country}' and taxon=='{taxon}' and {cohort_group} == '{cohort2}'",
            window_size=window_size,
            analysis="gamb_colu",
            min_cohort_size=10,
            max_cohort_size=100
    )
    return pos, xp_ehh # x: ndarray xp_ehh: ndarray
  except Exception as e:
    print(f"Error running XP-EHH: {e}")
    return None, None

In [ ]:
# Initialize an empty DataFrame to store results
final_xp_ehh_df = pd.DataFrame()
taxon1 = 'bissau'
taxon2 = 'gambiae'
#contigs = ['2L', '2R', '3L', '3R', 'X']
contigs=["3R"]
country = 'Gambia, The'  # Ensure the country variable is defined
window_size = 100  # Define the window size
cohort_results = []  # Initialize an empty list to collect results

for contig in contigs:
    print(f"Processing contig: {contig}")

    # Run the genome-wide scan for selection (XP-EHH)
    try:
        # Assuming run_XP_EHH returns positions and XP-EHH values
        pos, xp_ehh = run_XP_EHH(taxon1, taxon2, contig, country, window_size=window_size)
        # add an empty dimension to XP-EHH array if 1D
        xp_ehh = np.reshape(xp_ehh, (xp_ehh.shape[0], -1))
        for i in range(xp_ehh.shape[1]):
           xp_ehh_perc = xp_ehh[:, i]

        # Append results for the current contig to cohort_results
        cohort_results.append({
            "chrom": contig,
            "Chr_pos": pos,        # Chromosomal positions
            "xp_ehh": xp_ehh_perc,      # XP-EHH values
            "window_size": window_size,  # Window size used
            "taxon_1": taxon1,
            "taxon_2": taxon2,
            "country": country
        })
        print(f"XP-EHH scan completed for {contig}.")
    except Exception as e:
        print(f"Error running XP-EHH for {contig}: {e}")
        continue

# Process results and append to the final DataFrame
for result in cohort_results:
    temp_df = pd.DataFrame({
        "Chr_pos": result["Chr_pos"],
        "xp_ehh": result["xp_ehh"],
        "chrom": result["chrom"],
        "window_size": result["window_size"],
        "taxon_1": result["taxon_1"],
        "taxon_2": result["taxon_2"],
        "country": result["country"]
    })
    final_xp_ehh_df = pd.concat([final_xp_ehh_df, temp_df], ignore_index=True)

print("XP-EHH selection scan complete.")

# Save the final results to a CSV file
final_xp_ehh_df.to_csv(f'XP_EHH_{taxon1}_vs_{taxon2}_results.csv', index=False)
final_xp_ehh_df

##XP-EHH plot

In [ ]:
# List of unique chromosomes
final_xp_ehh_df = pd.read_csv('/content/XP_EHH_bissau_vs_gambiae_results.csv')
chromosomes = final_xp_ehh_df['chrom'].unique()

# Define color map for cohorts
cohorts = final_xp_ehh_df['taxon_1'].unique()  # Assuming cohorts are defined by `taxon_1`
colors = plt.cm.tab20.colors  # Color palette
color_map = [colors[i % len(colors)] for i in range(len(cohorts))]

# Determine subplot grid size
ncols = 2
nrows = (len(chromosomes) + 1) // ncols

# Create subplots
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, 7 * nrows), sharex=False, sharey=False)
axes = axes.flatten()  # Flatten the axes array for easy indexing

# Loop through each chromosome and create a plot
for i, chrom in enumerate(chromosomes):
    ax = axes[i]
    chr_data = final_xp_ehh_df.query(f"chrom == '{chrom}'")

    # Unique cohorts
    cohorts = chr_data['taxon_1'].unique()  # Assuming `taxon_1` defines cohorts
    x_ticks = []
    x_labels = []
    current_x = 0

    for idx, cohort in enumerate(cohorts):
        # Filter data for the current cohort
        cohort_data = chr_data[chr_data['taxon_1'] == cohort]

        # X values: Positions adjusted by current_x offset
        x = cohort_data['Chr_pos'] + current_x
        y = cohort_data['xp_ehh']  # Use XP-EHH values

        # Scatter plot for current cohort
        ax.scatter(x, y, color=color_map[idx], s=10, label=f'{cohort}')

        # Add ticks and labels for cohorts
        x_ticks.append(x.median())  # Use median position for tick
        x_labels.append(f'{cohort}')

        # Update current_x for the next cohort
        current_x += cohort_data['Chr_pos'].max() + 50  # Add spacing between cohorts

    # Highlight the threshold line
    """threshold = chr_data['xp_ehh'].quantile(0.99)  # Top 1% as threshold
    ax.axhline(y=threshold, color='black', linestyle='--', linewidth=1)"""

    # Set title for the subplot
    ax.set_title(f'XP-EHH Plot for Chromosome {chrom}', fontsize=12)
    ax.set_ylabel('XP-EHH')
    ax.legend(loc='upper right', fontsize=8)

    # Set x-axis ticks and labels
    ax.set_xticks(ticks=x_ticks)
    ax.set_xticklabels(labels=x_labels, rotation=45, ha='right')

# Remove empty subplots if the number of chromosomes is not a multiple of 2
for j in range(len(chromosomes), len(axes)):
    fig.delaxes(axes[j])

# Set global x-axis label
fig.supxlabel('Cohorts', fontsize=14)

# Adjust layout
plt.tight_layout()

# Save the plot
plt.savefig('xp_ehh_per_cohort.png', dpi=300)

# Display the plot
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load XP-EHH result
df = pd.read_csv("/content/XP_EHH_bissau_vs_gambiae_results.csv", sep=",")  # Adjust separator
df
# Ensure proper types
df = df.dropna(subset=['chrom', 'Chr_pos', 'xp_ehh'])
df['chrom'] = df['chrom'].astype(str)

# Convert chromosome to ordered category if numeric
try:
    df['chrom'] = pd.Categorical(df['chrom'], ordered=True, categories=sorted(df['chrom'].unique(), key=lambda x: int(x)))
except:
    pass

# Plot
plt.figure(figsize=(14, 6))
colors = ['skyblue', 'steelblue']
for i, (chrom, group) in enumerate(df.groupby('chrom')):
    plt.scatter(group['Chr_pos'], group['xp_ehh'],
                s=8, label=f'Chr {chrom}', alpha=0.6, color=colors[i % 2])

"""# Optional threshold line
plt.axhline(2, color='red', linestyle='--', lw=1)
plt.axhline(-2, color='red', linestyle='--', lw=1)"""

# Axis and labels
plt.xlabel("Position (bp)", fontsize=12)
plt.ylabel("XP-EHH Score", fontsize=12)
plt.title("XP-EHH per Chromosome", fontsize=14)
plt.legend(title='Chromosome', bbox_to_anchor=(1.01, 1), loc='upper left', fontsize=8)
plt.tight_layout()
plt.grid(alpha=0.3)

plt.show()


In [ ]:
taxon1 = 'bissau'
taxon2 = 'gambiae'
contig = '3R'
country = 'Gambia, The'
window_size = 200
gene_labels = ['Or13', 'Or15', 'Or16', 'Or17', 'Or30', 'Or46', 'Or47', 'Or53', 'Or55']
ag3.plot_xpehh_gwss(
            contig=contig,
            cohort1_query = f"country=='{country}' and taxon=='{taxon1}'",
            cohort2_query = f"country=='{country}' and taxon=='{taxon2}'",
            window_size=window_size,
            analysis="gamb_colu",
            max_cohort_size=100,
)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from matplotlib.colors import TwoSlopeNorm
import warnings
warnings.filterwarnings('ignore')

# Define custom colors for population comparisons
COMPARISON_COLORS = {
    'gambiae_vs_coluzzii': 'blue',
    'gambiae_vs_bissau': 'purple', 
    'coluzzii_vs_bissau': 'orange',
    'default': 'gray'
}


def plot_manhattan_style_xp_ehh(df, save_path=None, show_absolute=False, annotations=None):
    """
    Create Manhattan plot style visualization for XP-EHH data with optional annotations.
    
    Parameters:
    df: DataFrame with XP-EHH data
    save_path: Path to save the plot
    show_absolute: If True, plot absolute values; if False, plot signed values
    annotations: List of tuples (label, chrom, pos, text_x, text_y, y_text_offset)
    """
    # --- Compute chromosome offsets ---
    chr_offsets = {}
    cumulative_bp = 0
    for chrom in sorted(df['chrom'].unique()):
        min_pos = df[df['chrom'] == chrom]['Chr_pos'].min()
        chr_offsets[chrom] = cumulative_bp
        chrom_max = df[df['chrom'] == chrom]['Chr_pos'].max()
        cumulative_bp += chrom_max - min_pos + 1  # to avoid overlap

    # --- Create subplots ---
    fig, axes = plt.subplots(len(df['chrom'].unique()), 1,
                             figsize=(16, 4 * len(df['chrom'].unique())),
                             sharex=False)

    if len(df['chrom'].unique()) == 1:
        axes = [axes]

    for i, chrom in enumerate(sorted(df['chrom'].unique())):
        chrom_data = df[df['chrom'] == chrom]

        for comparison in chrom_data['comparison'].unique():
            comp_data = chrom_data[chrom_data['comparison'] == comparison]

            color = COMPARISON_COLORS.get(comparison, COMPARISON_COLORS['default'])
            y_values = comp_data['abs_xp_ehh'] if show_absolute else comp_data['xp_ehh']

            # Scatter plot
            axes[i].scatter(
                comp_data['Chr_pos'], y_values,
                c=color, alpha=0.7, s=20, label=f'{comparison}'
            )

        # --- Add annotations for this chromosome ---
        if annotations:
            for label, ann_chrom, pos, text_x, text_y, y_text_offset in annotations:
                if ann_chrom == chrom and chrom in chr_offsets:
                    # Adjust to chromosome-local plot coordinates
                    genome_pos = pos
                    text_x_adj = text_x

                    axes[i].annotate(
                        label,
                        xy=(genome_pos, text_y),
                        xytext=(text_x_adj, text_y + y_text_offset),
                        color='darkred',
                        fontsize=14,
                        arrowprops=dict(arrowstyle="->", color='slategrey')
                    )

        axes[i].set_title(f'Chromosome {chrom} - Population Differentiation (XP-EHH)')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)

    axes[-1].set_xlabel('Genomic Position')
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()


def process_xp_ehh_data(xp_ehh_df):
    """
    Process XP-EHH data for plotting
    
    Parameters:
    xp_ehh_df: DataFrame with columns ['Chr_pos', 'xp_ehh', 'chrom', 'window_size', 'taxon_1', 'taxon_2']
    """
    # Make a copy to avoid modifying original
    df = xp_ehh_df.copy()

    # Ensure proper column names (handle any spacing issues)
    df.columns = df.columns.str.strip()

    # Convert scientific notation if needed
    if 'Chr_pos' in df.columns:
        df['Chr_pos'] = df['Chr_pos'].astype(float).astype(int)

    # Create comparison labels
    df['comparison'] = df['taxon_1'] + '_vs_' + df['taxon_2']

    # Calculate absolute XP-EHH for significance assessment
    df['abs_xp_ehh'] = np.abs(df['xp_ehh'])

    # Define selection direction
    df['selection_population'] = df.apply(
        lambda row: row['taxon_1'] if row['xp_ehh'] > 0 else row['taxon_2'], 
        axis=1
    )

    # Define significance thresholds
    df['selection_significance'] = pd.cut(df['abs_xp_ehh'],
                                        bins=[0, 2, 2.5, float('inf')],
                                        labels=['Non-significant', 'Moderate', 'Strong'])

    return df

def identify_xp_ehh_hotspots(df, threshold=2.5):
    """
    Identify and report XP-EHH selection hotspots
    """
    hotspots = df[df['abs_xp_ehh'] > threshold].copy()

    print(f"XP-EHH Selection Hotspots (|XP-EHH| > {threshold}):")
    print(f"Total hotspots found: {len(hotspots)}")
    print("\nTop 10 strongest differentiation signals:")

    top_signals = hotspots.nlargest(10, 'abs_xp_ehh')[
        ['chrom', 'Chr_pos', 'xp_ehh', 'abs_xp_ehh', 'comparison', 'selection_population']
    ]
    print(top_signals.to_string(index=False))

    # Hotspots by chromosome and comparison
    print(f"\nHotspots by chromosome and population comparison:")
    hotspot_summary = hotspots.groupby(['chrom', 'comparison']).agg({
        'abs_xp_ehh': ['count', 'mean', 'max'],
        'Chr_pos': ['min', 'max']
    }).round(3)

    print(hotspot_summary)

    # Selection direction summary
    print(f"\nSelection direction summary:")
    direction_summary = hotspots.groupby(['comparison', 'selection_population']).size().unstack(fill_value=0)
    print(direction_summary)

    return hotspots

def plot_xp_ehh_distribution(df, save_path=None):
    """
    Plot distribution of XP-EHH values by population comparison
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # Distribution plot
    for comparison in df['comparison'].unique():
        comp_data = df[df['comparison'] == comparison]
        color = COMPARISON_COLORS.get(comparison, COMPARISON_COLORS['default'])
        
        axes[0].hist(comp_data['xp_ehh'], bins=50, alpha=0.6, 
                    label=comparison, color=color, density=True)

    axes[0].axvline(x=0, color='black', linestyle='-', alpha=0.5)
    axes[0].axvline(x=2, color='orange', linestyle='--', alpha=0.7)
    axes[0].axvline(x=-2, color='orange', linestyle='--', alpha=0.7)
    axes[0].axvline(x=2.5, color='red', linestyle='--', alpha=0.7)
    axes[0].axvline(x=-2.5, color='red', linestyle='--', alpha=0.7)
    axes[0].set_xlabel('XP-EHH')
    axes[0].set_ylabel('Density')
    axes[0].set_title('Distribution of XP-EHH Values')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Box plot by comparison
    comparison_data = [df[df['comparison'] == comp]['xp_ehh'] for comp in df['comparison'].unique()]
    box_plot = axes[1].boxplot(comparison_data, labels=df['comparison'].unique(), patch_artist=True)
    
    # Color the boxes
    for patch, comparison in zip(box_plot['boxes'], df['comparison'].unique()):
        color = COMPARISON_COLORS.get(comparison, COMPARISON_COLORS['default'])
        patch.set_facecolor(color)
        patch.set_alpha(0.7)

    axes[1].axhline(y=0, color='black', linestyle='-', alpha=0.5)
    axes[1].axhline(y=2, color='orange', linestyle='--', alpha=0.7)
    axes[1].axhline(y=-2, color='orange', linestyle='--', alpha=0.7)
    axes[1].axhline(y=2.5, color='red', linestyle='--', alpha=0.7)
    axes[1].axhline(y=-2.5, color='red', linestyle='--', alpha=0.7)
    axes[1].set_ylabel('XP-EHH')
    axes[1].set_title('XP-EHH Distribution by Population Comparison')
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()


In [ ]:
annotations = [
    ('$vgsc$ region', '2L', 2431617, 600000, 0.5, 0.2),
    ('$Gste$ region', '3R', 28500000, 28600000, 1.9, 0.5),
    ('$Cyp6aa/Cyp6p$', '2R', 28500000, 28600000, 0.5, 0.35),
    ('$ORs$ region', '3R', 32000000, 33080000, -1.26, -0.5),
    ('$ORs$ region', '3R', 25747232, 20000000, -2, -0.5),
    ('$CYPK9$ region', 'X', 15000000, 15010000, 0.5, 0.25),
    ('$COEAE2F$', '2L', 28500000, 28600000, 0.18, 0.25),
    ('E3 ubiquitin-protein\n fumarylacetoacetate ','2L', 22281000,22283000, 0.40, 0.5),
    ('AGP002284', '2R', 18570000, 18600000, 0.5, 0.25)
]
# Load your XP-EHH data
xp_ehh_data = pd.read_csv('XP_EHH_coluzzii_vs_bissau_results.csv')
#xp_ehh_data = xp_ehh_data.query("chrom == '3R'")

# Process the data
processed_data = process_xp_ehh_data(xp_ehh_data)
# Call with annotation
plot_manhattan_style_xp_ehh(processed_data, show_absolute=False, annotations=annotations)


The XP-EHH on 3R chrom shows 2 to 3 pic corresponding to difference in term on selection between Bissau molecular form and An. gambiae. In that list we have the 28.5bk region containing Gste genes which seems to be particular selected in An. gambiae population while the the **32kb where Ors genes** are located seems to be selected in Bissau molecular form. Another pic found around 25.74kb where Or29 is located is only selected in Bissau molecular form.
 This result confirms our assumption about independant evolution of Ors genes regions in Bissau molecular form and concorce with IBD result where no share IBD was found around ors genes region between An. gambiae complex.